### Data620 - HW5- Part2 - Document Classifier
### Team6 :Santosh Manjrekar, Dhananjay Kumar ,Sang Yoon(Andy)Hwang, Matheesha Thambeliyagodage

### Assignment

It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam. 

Here is one example of such data: http://archive.ics.uci.edu/ml/datasets/Spambase

spambase.zip file was downloaded from the above siteand is checked into github along with this codebase

spambase.names file contains the list of  feature names or attribute names listed for each message in spambase.data file

spambase.data contains actual data for each message

### Data Summary :

#### Number of Instances: 4,601 (1,813 Spam = 39.4%)

#### Number of Attributes: 58 (57 continuous, 1 nominal class label)

#### Missing Attribute Values: "None"

#### Class Distribution:
#### Spam: 1,813 (39.4%)    Non-Spam: 2,788 (60.6%)



### SPAM Document Classifier using Naive Bayes methode

In [125]:
import numpy as np
from sklearn.model_selection import train_test_split

In [126]:
datafile = open('data/spambase.data','r')
data = []
for line in datafile:
    line = [float(element) for element in line.rstrip('\n').split(',')]
    data.append(np.asarray(line))

In [127]:
num_features = 48
### extract features dataset
X = [data[i][:num_features] for i in range(len(data))]
### extract classification 1= spam 0= ham or no spam
y = [int(data[i][-1]) for i in range(len(data))]

In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [129]:
#Making likelihood estimations

#Find the two classes
X_train_class_0 = [X_train[i] for i in range(len(X_train)) if y_train[i]==0]
X_train_class_1 = [X_train[i] for i in range(len(X_train)) if y_train[i]==1]

#Find the class specific likelihoods of each feature
likelihoods_class_0 = np.mean(X_train_class_0, axis=0)/100.0
likelihoods_class_1 = np.mean(X_train_class_1, axis=0)/100.0

In [130]:
#Calculate the class priors
num_class_0 = float(len(X_train_class_0))
num_class_1 = float(len(X_train_class_1))

prior_probability_class_0 = num_class_0 / (num_class_0 + num_class_1)
prior_probability_class_1 = num_class_1 / (num_class_0 + num_class_1)

log_prior_class_0 = np.log10(prior_probability_class_0)
log_prior_class_1 = np.log10(prior_probability_class_1)

In [131]:
def calculate_log_likelihoods_with_naive_bayes(feature_vector, Class):
    assert len(feature_vector) == num_features
    log_likelihood = 0.0 #using log-likelihood to avoid underflow
    if Class==0:
        for feature_index in range(len(feature_vector)):
            if feature_vector[feature_index] == 1: #feature present
                log_likelihood += np.log10(likelihoods_class_0[feature_index]) 
            elif feature_vector[feature_index] == 0: #feature absent
                log_likelihood += np.log10(1.0 - likelihoods_class_0[feature_index])
    elif Class==1:
        for feature_index in range(len(feature_vector)):
            if feature_vector[feature_index] == 1: #feature present
                log_likelihood += np.log10(likelihoods_class_1[feature_index]) 
            elif feature_vector[feature_index] == 0: #feature absent
                log_likelihood += np.log10(1.0 - likelihoods_class_1[feature_index])
    else:
        raise ValueError("Class takes integer values 0 or 1")
        
    return log_likelihood

In [132]:
def calculate_class_posteriors(feature_vector):
    log_likelihood_class_0 = calculate_log_likelihoods_with_naive_bayes(feature_vector, Class=0)
    log_likelihood_class_1 = calculate_log_likelihoods_with_naive_bayes(feature_vector, Class=1)
    
    log_posterior_class_0 = log_likelihood_class_0 + log_prior_class_0
    log_posterior_class_1 = log_likelihood_class_1 + log_prior_class_1
    
    return log_posterior_class_0, log_posterior_class_1

In [133]:
def classify_spam(document_vector):
    feature_vector = [int(element>0.0) for element in document_vector]
    log_posterior_class_0, log_posterior_class_1 = calculate_class_posteriors(feature_vector)
    if log_posterior_class_0 > log_posterior_class_1:
        return 0
    else:
        return 1

In [134]:
#Predict spam or not on the test set
predictions = []
for email in X_test:
    predictions.append(classify_spam(email))

In [135]:
def evaluate_performance(predictions, ground_truth_labels):
    correct_count = 0.0
    for item_index in range(len(predictions)):
        if predictions[item_index] == ground_truth_labels[item_index]:
            correct_count += 1.0
    accuracy = correct_count/len(predictions)
    return accuracy

In [136]:
accuracy_of_naive_bayes = evaluate_performance(predictions, y_test)
print(accuracy_of_naive_bayes)

0.8922675933970461


In [137]:
len(y_test)
from sklearn.metrics import confusion_matrix
print (confusion_matrix(y_test, predictions))

[[576 100]
 [ 24 451]]


#### So Naive Bayes classifier has an accuracy of 89.22%. 
Out of 676 non spam messages,it miscategorised 100 messages(error rate of 14.79%). 
Out of 475 spam messages ,it miscategorised 24 messages (error rate of 5.05%)


### SPAM Document Classifier using Random Forest algorithm

In [138]:
import nltk
import numpy as np
import pandas as pd
%matplotlib inline

# pull in the spam dataset
spam = pd.read_csv("data/spambase.csv")

In [139]:
# Summary stats
spam.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,SPAM_CLASS
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [140]:
# Variable types
print(spam.dtypes)

word_freq_make                float64
word_freq_address             float64
word_freq_all                 float64
word_freq_3d                  float64
word_freq_our                 float64
word_freq_over                float64
word_freq_remove              float64
word_freq_internet            float64
word_freq_order               float64
word_freq_mail                float64
word_freq_receive             float64
word_freq_will                float64
word_freq_people              float64
word_freq_report              float64
word_freq_addresses           float64
word_freq_free                float64
word_freq_business            float64
word_freq_email               float64
word_freq_you                 float64
word_freq_credit              float64
word_freq_your                float64
word_freq_font                float64
word_freq_000                 float64
word_freq_money               float64
word_freq_hp                  float64
word_freq_hpl                 float64
word_freq_ge

In [141]:
# Count spam and non-spam
count_spam = len(spam[spam.SPAM_CLASS==1])
count_nonspam = len(spam[spam.SPAM_CLASS==0])

print("Spam: %d" %count_spam)
print("Non-spam: %d" %count_nonspam)

Spam: 1813
Non-spam: 2788


##### Split data into two datasets: training & testing

In [142]:
# Split into test, train, validate
#percTrain = 0.7
#percVal = 0.15
#percTest = 0.15
#N = len(spam)
#trainNum = int(percTrain * N)
#valNum = int(percVal * N)
#testNum = N - trainNum - valNum

X = spam.values[:, 0:57]
Y = spam.values[:, 57]

# Splitting the data into training and test set
# X_train and Y_train are for training
# X_test and Y_test are for testing

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

In [143]:
# Check lengths
print ("Training set: %d" %len(X_train))
print ("Testing set: %d" %len(X_test))
#print ("Testing set: %d" %len(testSet))
print ("Total: %d" %(len(X_train) + len(X_test) ))

Training set: 3220
Testing set: 1381
Total: 4601


### Randon Forest

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html



In [144]:
# Random forest - train
from sklearn import ensemble
from sklearn.metrics import accuracy_score

rf = ensemble.RandomForestClassifier(criterion="entropy", random_state=88)
rf_fit = rf.fit(X_train, Y_train)

Y_predict = rf_fit.predict(X_test)
accuracy_score(Y_test, Y_predict)

0.9333816075307748

Random Forest method has 93.33% accuracy in classifying emails.

In [145]:
from sklearn import metrics
print(metrics.classification_report(Y_predict, Y_test))

             precision    recall  f1-score   support

        0.0       0.96      0.93      0.94       843
        1.0       0.89      0.94      0.92       538

avg / total       0.94      0.93      0.93      1381



In [146]:
important_features_dict = {}
for x,i in enumerate(rf.feature_importances_):
    important_features_dict[x]=i


important_features_list = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

##print ('Most important features: %s' %important_features_list)

cols = spam.columns

#Print top 10 features contributing to spam messages

print ('Top 10 features contributing to decide spam message:')
for x in range(10):
    print(cols[important_features_list[x]])




Top 10 features contributing to decide spam message:
char_freq_!
char_freq_$
word_freq_free
capital_run_length_longest
capital_run_length_average
word_freq_remove
word_freq_george
word_freq_your
word_freq_hp
capital_run_length_total


#### Conclusion :

Random Forest method has more accuracy than Naive Bays method in classifying messages accurately.

Following characters or words  contribute more in deciding spam messages 

!,$,free etc...
